# Tracking AR 12970 and creating an ndcube

In this example we will attempt to find AR 12790, download HMI and AIA 193 data for it, track the AR in carrington coordinates and then create an animation of a composite image.


To install ndcube run: `pip install "ndcube>=2.0.0b1"`

In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.wcs import WCS
from reproject import reproject_adaptive, reproject_interp

import sunpy.map
import sunpy.coordinates
from sunpy.net import Fido, attrs as a, hek

from ndcube import NDCube

## Obtaining the Data

Let's start by finding a reference coord for AR 12790

In [ ]:
hclient = hek.HEKClient()

In [ ]:
res = hclient.search(hek.attrs.Time("2020/11/01", "2020/12/16"), hek.attrs.AR, hek.attrs.AR.NOAANum == 12790)

In [ ]:
res['ar_noaanum', 'event_starttime', 'hgc_coord']

In [ ]:
res['hgc_coord'] = SkyCoord(np.float64([c.strip("POINT()").split() for c in res['hgc_coord']]), unit=u.deg, frame="heliographic_carrington")

In [ ]:
res['event_starttime'] = Time(res['event_starttime'])

In [ ]:
ar12790 = res['ar_noaanum', 'event_starttime', 'hgc_coord']

In [ ]:
ar12790

In [ ]:
start_time, end_time = ar12790['event_starttime'][[0,-1]]

In [ ]:
start_time, end_time

In [ ]:
hmi = a.Instrument.hmi & a.Physobs.los_magnetic_field
aia = a.Instrument.aia & a.Wavelength(19.3*u.nm)

In [ ]:
results = Fido.search(a.Time(start_time, end_time) & a.Sample(1*u.day), hmi | aia)

In [ ]:
results

In [ ]:
files = Fido.fetch(results[:, :11], path="~/psi/12790/{instrument}")

In [ ]:
files

## Processing the Data

Now we have got the data let's take the HMI images and reproject them into carrington coordinates so we can clearly see the evolution of the feature.

In [ ]:
%matplotlib widget

In [ ]:
ar12790 = ar12790[:11]

In [ ]:
hmi_maps = sunpy.map.Map("~/psi/12790/HMI/*")
aia_maps = sunpy.map.Map("~/psi/12790/AIA/*")

In [ ]:
data = [[h.observer_coordinate.lon, h.observer_coordinate.lat, h.observer_coordinate.radius] for h in hmi_maps]

In [ ]:
observers = SkyCoord(data, unit=(u.deg, u.deg, u.m), obstime=[h.date for h in hmi_maps], frame="heliographic_stonyhurst")

In [ ]:
ar12790['hgc_coord'] = SkyCoord(ar12790['hgc_coord'], observer=observers, obstime=ar12790['event_starttime'])

In [ ]:
plt.figure()
ax = plt.subplot(projection=hmi_maps[0])
hmi_maps[0].plot(vmin=-1500, vmax=1500, cmap='hmimag')
ax.plot_coord(ar12790['hgc_coord'], "wo")
plt.colorbar()

In [ ]:
def make_carrington_wcses(maps, lat_scale, lon_scale):
    lat_scale = 10
    lon_scale = 10
    shape = (180*lat_scale, 360*lon_scale)

    wcses = []
    for amap in maps:
        observer = amap.observer_coordinate
        header = sunpy.map.make_fitswcs_header(
            shape,
            SkyCoord(0, 0, unit=u.deg, observer=observer, obstime=amap.date, frame="heliographic_carrington"),
            scale=[1/lon_scale, 1/lat_scale]*u.deg/u.pix,
            projection_code='CAR'
        )
        wcs = WCS(header)
        wcs.heliographic_observer = in_map.observer_coordinate
        wcses.append(wcs)
        
    return wcses

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
hmi_carrington_wcses = make_carrington_wcses(hmi_maps, 20, 20)

In [ ]:
out_maps = []
for in_map, out_wcs in tqdm_notebook(zip(hmi_maps, hmi_carrington_wcses),
                                     total=len(hmi_maps)):
    array, _ = reproject_interp(in_map, out_wcs, shape_out=shape)
    out_maps.append(sunpy.map.Map(array, out_wcs))

## Making a NDCube

Now we have the data on a shared grid, let's build a NDCube out of the results, with a 3D WCS and then experiment a little with what NDCube can provide.

In [ ]:
seq = sunpy.map.Map(out_maps, sequence=True)

In [ ]:
times = Time([m.date for m in seq])
time_delta = times[1:] - times[:-1]
time_delta.to(u.day).round()

In [ ]:
cube_header = copy.copy(seq[0].meta)

In [ ]:
cube_header

In [ ]:
cube_header['wcsaxes'] = 3
cube_header['CTYPE3'] = "UTC"
cube_header['CDELT3'] = time_delta[0].to_value(u.s)
cube_header['CRVAL3'] = 0
cube_header['CRPIX3'] = 0
cube_header['CUNIT3'] = "s"
cube_header['CNAME1'] = "Carrington Longitude"
cube_header['CNAME2'] = "Carrington Latitude"
cube_header['CNAME3'] = "Time"
cube_header['DATEREF'] = times[0].isot
del cube_header['mjdref']
del cube_header['mjd-obs']

In [ ]:
cube_header

In [ ]:
cube_wcs = WCS(cube_header)

In [ ]:
cube_wcs

In [ ]:
cube_wcs.pixel_to_world(0, 0, 0)

In [ ]:
data = seq.as_array()

In [ ]:
data.shape

In [ ]:
data = np.rollaxis(data, -1)

In [ ]:
data.shape

In [ ]:
ndc = NDCube(data, wcs=cube_wcs)

In [ ]:
ndc.plot(vmin=-1500, vmax=1500, cmap='hmimag')

In [ ]:
ndc.array_axis_physical_types

In [ ]:
small_cube = ndc.crop_by_values([240, -33, None], [260, -13, None], units=(u.deg, u.deg, u.s))

In [ ]:
small_cube.plot(vmin=-1500, vmax=1500)

In [ ]:
small_cube.axis_world_coords()

In [ ]:
small_cube.axis_world_coords("time")